## Offline Testing  
  
Sandbox for the testing of the deployed app without using any Streamlit commands to allow for line-by-line debugging.  

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
import re

**Lambda Functions**

In [2]:
def quantity_replacement(x):
    quantity = x
    try:
        quantity = float(quantity)
    except:
        quantity = 0
    return quantity

def names_cleaning(x):
    x = x.upper()
    x = re.sub(r'\([^)]*\)', '', x)
    sevenup_exception = re.search('7 UP|7UP', x)
    if sevenup_exception:
        x = x
    else:
        x = re.sub("[^a-zA-ZéÉíÍóÓúÚáÁ ]+", "", x)
        x = ' '.join( [w for w in x.split() if len(w)>2] )
    return x

# convert quantities to ml and gr
def quantity_conversion(x):
    thousand_multiplier = ['KG', 'K', 'KS', 'KGS', 'LTR', 'LT', 'LIT', 'LITRE', 'LITER']
    ounce_multiplier = ['OZ']
    alc_multiplier = ['CL']
    cup_multiplier = ['CUP', 'CUPS']
    spoon_multiplier = ['TBLS', 'TBSP']
    soda_multiplier = ['CAN', 'BTL']
    if x['Unit of Measurement'] in thousand_multiplier:
        m = float(x['Quantity']) * 1000
    elif x['Unit of Measurement'] in ounce_multiplier:
        m = float(x['Quantity']) * 30
    elif x['Unit of Measurement'] in alc_multiplier:
        m = float(x['Quantity']) * 10
    elif x['Unit of Measurement'] in cup_multiplier:
        m = float(x['Quantity']) * 237
    elif x['Unit of Measurement'] in spoon_multiplier:
        m = float(x['Quantity']) * 15
    elif x['Unit of Measurement'] in soda_multiplier:
        m = float(x['Quantity']) * 300
    else:
        m = x['Quantity']
    return m

def multiplier_search(s):
    m = 1
    multiplier = re.search('[0-9]+[xX]| [0-9]+ [xX] | [xX] [0-9]+ | [xX][0-9+] ', s)
    if multiplier:
        interim = multiplier.group()
        interim = re.sub('[^0-9]+', "", interim)
        m = float(interim)
    return m

def unit_search(s):
    m = 1000
    s = re.sub(',', '.', s)
    common_uoms = ['[0-9]+ML', ' [0-9]+ ML', 
                   ' [0-9]+ C', '[0-9]+C', '[0-9]+CS', '\-[0-9.]+ ML', ' [0-9]+ CS',
                   '[0-9.,]+KG', ' [0-9.,]+ KG', '\-[0-9,.]+KG', '[0-9.,]+KGS', ' [0-9.,]+ KGS', '\-[0-9.,]+KGS', '[0-9.,]+KS', ' [0-9.,]+ KS',
                   '[0-9]+GR', ' [0-9]+ GR ', '[0-9]+ GR//', ' [0-9]+ GR//', '[0-9]+GMS', ' [0-9.]+ GMS', '[0-9 ]+GM', ' [0-9 ]+ GM', '[0-9]+G', ' [0-9]+ G ', '[0-9\-]+GRAMS',  ' [0-9]+ GRAMS',                        
                   '[0-9]+LB', ' [0-9]+ LB',
                   '[0-9.]+CL', ' [0-9.]+ CL',
                   '[0-9.]+LTR', ' [0-9.]+ LTR', '[0-9.]+LT','[0-9.]+L', ' [0-9.]+ LT',' [0-9.]+ L',
                   '[0-9]+GAL', ' [0-9]+ GAL', '[0-9]+GL',
                   '[0-9.]+OZ', ' [0-9.]+ OZ'
                  ]
    thousand_multiplier = ['KG', 'KGS', 'KS', 'L', 'LT', 'LTR']
    gal_multiplier = ['GAL', 'GL']
    ounce_multiplier = ['OZ']
    lb_multiplier = ['LB']
    alcohol = ['CL']
    alcohol2 = ['C']
    unit = re.search('|'.join(common_uoms), s)
    if unit:
        interim = unit.group()
        interim = re.sub('[^0-9.]+', "", interim)
        uom = re.sub('[^a-zA-Z]+', "", unit.group())
        if interim != '.':
            if uom in alcohol:
                interim = float(interim) * 10
            elif uom in thousand_multiplier:
                interim = float(interim) * 1000
            elif uom in gal_multiplier:
                interim = float(interim) * 3780
            elif uom in ounce_multiplier:
                interim = float(interim) * 30
            elif uom in lb_multiplier:
                interim = float(interim) * 454
            elif uom in alcohol2:
                interim = float(interim) * 10
            m = float(interim)
            if m == 0:
                m = 1000
    return m

def soda_multiplier(t):
    sec_multiplier = 1
    soda_identifier = ' CS'
    soda_search = re.search(soda_identifier, t)
    if soda_search:
        sec_multiplier = 24
    return sec_multiplier  

def drinks_unit_price_multiplier(d):
    multiplier = d['Unit Price']
    if d['Upload Time'] < '2021-11-22' and (d['Category'] == 'Alcoholic Beverage' or d['Category'] == 'Beverage - Alcohol' or d['Category'] == 'Alcohol Beverage' or d['Category']== 'Alc Beverage'):
        multiplier = 1.3 * d['Unit Price']
    elif d['Upload Time'] < '2021-11-22' and (d['Category'] == 'Non Alcoholic Beverage' or d['Category'] == 'Beverage - Non Alcohol' or d['Category'] == 'Beverage - Soft' or d['Category'] == 'Beverage  - Soft' or d['Category'] == 'N-Alc Beverage'):
        multiplier_exclusion = re.search('JUICE|SYRUP|AQU|SPARKLING', d['Product Name'])
        if multiplier_exclusion:
            multiplier = 1 * d['Unit Price']
        else:
            multiplier = 1.5 * d['Unit Price']
    return multiplier  

def unit_price_adjustment(x):
    x = re.sub(r'AED ', '', x)
    x = re.sub(r',', '', x)
    x = float(x)
    return x

def load_transformer():
    sbert_model = SentenceTransformer('stsb-mpnet-base-v2')
    return sbert_model

def recipe_item_embeddings_fn(unique_recipe_items):
    recipe_item_embeddings = sbert_model.encode(unique_recipe_items, convert_to_tensor = True)
    return recipe_item_embeddings

def stock_in_embeddings_fn(recipe_ingredients_list):
    stock_in_embeddings = sbert_model.encode(recipe_ingredients_list, convert_to_tensor = True)
    return stock_in_embeddings

def unit_dict_accretion(x):
    uploaded_unit = x['Corrected Unit Size']
    if uploaded_unit == 0:
        uploaded_unit = x['Unit Size']
    return uploaded_unit

In [3]:
# init. sbert
sbert_model = load_transformer()

**Handling POS Data**

In [6]:
 # import pos data
pos_sheet_df = pd.read_excel(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\POS.xlsx', skiprows = 6, sheet_name = "Revenue per article")
pos_sheet_df = pos_sheet_df.dropna(how = 'all')
all_pos_cleaned = pos_sheet_df.loc[(pos_sheet_df['Article']!= 'Total'),].copy()
all_pos_cleaned['Article'] = all_pos_cleaned.loc[:,'Article'].str.upper()

# total revenue
all_pos_cleaned = all_pos_cleaned.rename(columns = {'Net revenue': 'Revenue'})
all_revenue = round(all_pos_cleaned['Revenue'].sum(),2)

C:\Users\wesch\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Handling Recipe Data**

In [ ]:
# import recipe data
recipe_sheet_df = pd.read_excel(r'C:\Users\wesch\Documents\FoodRazor\expo\output\nov21_reporting\Scarpetta\Recipe.xlsx', sheet_name = 'Reformatted')
recipe_sheet_df = recipe_sheet_df.dropna(how = 'all')
recipe_sheet_df = recipe_sheet_df.dropna(subset = ['Food Item (As per POS system)'])
recipe_sheet_df = recipe_sheet_df.dropna(subset = ['Ingredient Ordered (if known)'])
recipe_sheet_df['Food Item (As per POS system)'] = recipe_sheet_df['Food Item (As per POS system)'].apply(lambda x: x.rstrip())

# ensure servings column is an int, upper case all and clean the ingredient names
# check and replace quantity column to make sure it is an integer

recipe_sheet_df['Quantity'] = recipe_sheet_df.Quantity.apply(lambda x: quantity_replacement(str(x)))
    
recipe_sheet_df = recipe_sheet_df.assign(
    Servings = lambda x: x.Servings.astype(int),
    Quantity = lambda y: y.Quantity.astype(float)/y.Servings
)

# upper case all and clean the ingredient names
recipe_sheet_df.replace(np.inf, 0, inplace = True)
recipe_sheet_df['Food Item (As per POS system)'] = recipe_sheet_df.loc[:,'Food Item (As per POS system)'].str.upper()
recipe_sheet_df['Unit of Measurement'] = recipe_sheet_df.loc[:,'Unit of Measurement'].str.upper()        
recipe_sheet_df['Ingredient'] = recipe_sheet_df['Ingredient Ordered (if known)'].apply(lambda x: names_cleaning(str(x))) 

## rename new quantity column accordingly
recipe_sheet_df['NewQuantity'] = recipe_sheet_df.apply(quantity_conversion, axis = 1)
recipe_sheet_df.drop('Quantity', axis = 1, inplace = True)
recipe_sheet_df.rename(columns = {'NewQuantity': 'Quantity'}, inplace = True)
    
# get list of ingredients
recipe_ingredients_list = recipe_sheet_df['Ingredient'].drop_duplicates().tolist()

**POS to Recipe Items Ingredients Crosswalk**

In [ ]:
## get list of food items in recipe sheet
unique_recipe_items = recipe_sheet_df['Food Item (As per POS system)'].dropna().drop_duplicates().tolist()

# initialize embeddings
recipe_item_embeddings = recipe_item_embeddings_fn(unique_recipe_items)

# get unique menu items from POS
pos_items = all_pos_cleaned['Article'].dropna().drop_duplicates().tolist()
    
# get a list of most similar item on the menu from recipe and pos sheets
most_similar = []
for item in pos_items:
    query_embedding = sbert_model.encode(item, convert_to_tensor = True)
    cos_score = util.pytorch_cos_sim(query_embedding, recipe_item_embeddings)[0]
    best_match = torch.topk(cos_score, k = 1)
    for idx in best_match[1]:
        most_similar.append(unique_recipe_items[idx])

del query_embedding
    
# stacking into a df
matched_recipe_pos_df = pd.DataFrame({
    'POS Items': pos_items,
    'Recipe Items': most_similar
})

**Import Invoice Details Report**

In [ ]:
# import stock-in data
stock_in = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\InvoiceDetails_Amalgamated.csv', encoding = 'utf-8')

def unit_price_adjustment(x):
    x = re.sub(r'AED ', '', x)
    x = re.sub(r',', '', x)
    x = float(x)
    return x

start_date = '2021-12-01'
end_date = '2021-12-31'


df = stock_in.loc[(stock_in['Invoice Date'] >= start_date),].copy()
df = df.loc[(df['Invoice Date'] <= end_date),]

df['Subtotal'] = df['Subtotal'].apply(lambda x: unit_price_adjustment(str(x)))  

print(df['Subtotal'].sum())

In [28]:
# import stock-in data
stock_in = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\InvoiceDetails_Amalgamated.csv', encoding = 'utf-8')

# remove nas
stock_in = stock_in.dropna()

# remove trailing white spaces
stock_in['Category'] = stock_in['Category'].apply(lambda x: x.rstrip())

# remove those categories to be excluded
category_exclusion_split = ['Printing & Stationary', 'Printing and Stationery Supplies', 'Tax Adjustment', 'CAPEX', 'Other', 'Cleaning', 'Cleaning Supplies', 'Kitchen Supplies', 'Discount', 'Guest Supplies', 'Rounding', 'General Supplies', 
                            'Packaging',  'Bar Expenses','Operating Supplies General', 'HR', 'Payroll', 'Cleaning & Chemical', 'Utilities', 'Music & entertainment', 'Payroll & Related Expenses', 'PR & Marketing', 'Payroll Provision (Guest Chefs)',
                            'Transport', 'Accommodation & Air Tickets', 'OS&E - Kitchen', 'OS&E - FOH', 'Supplies Kitchen', 'Supplies Others', 'Supplies Cleaning', 'Provision', 'Accommodation', 'Crockery', 'Small Equipment', 
                            'Departmental Supplies', 'Cleaning', 'Disposables and Chemicals', 'Payroll and HR related',  'Staff Training', 'Paper Supplies', 'Uniforms', 'Passage', 'Travel Other', 'Miscellaneous Expenses', 
                            'VisaVisa MedicalsMedical Lev', 'Linen', 'Equipment Hire', 'Fuel',  'Meals', 'Marketing Expense', 'Managment Fee', 'Legal / Licenses', 'Pre-Opening Printing and Stationery Supplies', 'Pre-Opening - Kitchenware', 
                            'Pre-Opening - Payroll / HR Related', 'Pre-Opening - Accommodation & Air Tickets', 'Pre-Opening - Linen & Uniform', 'Pre-Opening - Legal / Licenses', 'Pre-Opening-PR & Marketing', 'Pre-Opening - IT & Technology', 
                            'Pre-Opening - China/Glass/Silver', 'Pre-Opening - Staff Meal', 'Pre-Opening Expenses', 'Pre-Opening Operating Supplies', 'Pre-Opening Training', 'Pre-Opening-PR & Marketing', 'Pre-Opening Music and Entertain. Expenses',
                            'OE-China', 'OE-Uniform', 'OE-Others', 'OE-Security/ Cleaning', 'OE-Kitchen supplies',  'OE-Music & entertainment', 'OE-Provision', 'FC-Bank Charges', 'OE - Admin - Supplies', 'OE-Glasswares', 'OE-Provision', 
                            'OE-Laundry', 'OE-Supply cleaning', 'OE-Packaging', 'OE-Guest supplies', 'OE-Printing & stationary', 'OE - ADMIN - Printing', 'OE-Others', 'FC-PR & Marketing', 'OE - Admin - Transport', 'FC-IT & Technology',
                            'OE-Bar Expenses', 'OE-Admin - Meal Allocation', 'Task force']
category_exclusion_split = [x.rstrip() for x in category_exclusion_split]
exclusions = ~stock_in.Category.isin(category_exclusion_split)
stock_in = stock_in[exclusions]

# remove trailing white spaces, excessive whitespaces and other stuff
stock_in['Product Name'] = stock_in['Product Name'].apply(lambda x: x.rstrip())

# extracting uoms and unit sizes
# need to convert everything from kg to g, and ltr to ml
common_uoms = ['GM', 'KG', '[0-9]+GR', ' GR ', 'GMS', 'KGS', '[0-9]+G', '[0-9.]+ GR//', 
               'GRAMS', ' GR ', ' G ', '[0-9.]+ GR', '[0-9.]+KS',  '[0-9.]+ GMS', '[0-9]+ KGS', 
               '[0-9]+LB', ' [0-9]+ LB', '[A-Z0-9/-]+KGS',
               'LTR', 'ML','[0-9]+CL', 'LT', '[0-9]+L',  '[0-9]+ML', ' [0-9.]+C', '[0-9.]+ CL', 
               '[0-9.]+ CS', '[0-9]+GAL', ' [0-9]+ GAL', '[0-9]+OZ', ' [0-9]+ OZ' ]

common_uoms_equivalent = ['GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 'GR', 
                          'GR', 'GR', 'GR', 'GR', 'GR', 'GR',
                          'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 'ML', 
                          'ML']

for uom in range(len(common_uoms)):
    stock_in.loc[stock_in['Product Name'].str.contains(common_uoms[uom]), 'Unit of Measurement'] = common_uoms_equivalent[uom]

stock_in['Unit'] = stock_in['Unit'].apply(lambda x: re.sub("KGS|KG", "GR", x))


# adjust unit price column
# look for multipliers, unit size, and apply
stock_in['Unit Price'] = stock_in['Unit Price'].apply(lambda x: unit_price_adjustment(str(x)))    
stock_in['Unit Price'] = stock_in.apply(drinks_unit_price_multiplier, axis = 1)
stock_in['multiplier'] = stock_in['Product Name'].apply(lambda x: multiplier_search(x))
stock_in['unit_size'] = stock_in['Product Name'].apply(lambda x: unit_search(x))
stock_in['soda_multiplier'] = stock_in['Product Name'].apply(lambda x: soda_multiplier(x))
stock_in = stock_in.assign(
   unit_size = lambda x: x.unit_size * x.multiplier * x.soda_multiplier
)
stock_in = stock_in.rename(columns = {'unit_size':'Unit Size'})
stock_in.drop(['multiplier', 'soda_multiplier'], axis = 1, inplace = True)    

# agg orders
stock_in_agg = stock_in[['Product Name', 'Qty', 'Unit', 'Unit Size',  'Unit Price']].copy()
stock_in_agg['Est Total Cost'] = stock_in_agg['Qty'] * stock_in_agg['Unit Price']
stock_in_agg.drop('Unit Price', axis = 1, inplace = True)
stock_in_agg = stock_in_agg.groupby(['Product Name', 'Unit', 'Unit Size']).sum()
stock_in_agg = stock_in_agg.reset_index()

# remove rows with empty values
stock_in_agg = stock_in_agg.loc[stock_in_agg['Product Name'] != '']

**Correct Unit Size Dictionary**

In [ ]:
matched_invoice_unit_sizes_df = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Baron\Output\Round 1\invoice_details_unit_size_amended.csv', encoding = 'utf-8')
matched_invoice_unit_sizes_df = matched_invoice_unit_sizes_df.fillna(0)

# replace unit size 
matched_invoice_unit_sizes_df = matched_invoice_unit_sizes_df.rename(columns = {'Unit Size': 'Corrected Unit Size'})
stock_in_agg = stock_in_agg.merge(matched_invoice_unit_sizes_df, on = 'Product Name', how = 'left')
stock_in_agg['Corrected Unit Size'] = stock_in_agg['Corrected Unit Size'].fillna(0)
stock_in_agg['Accreted Unit Size'] = stock_in_agg.apply(unit_dict_accretion, axis = 1)
stock_in_agg.drop(['Unit Size', 'Corrected Unit Size'], axis = 1, inplace = True)
stock_in_agg = stock_in_agg.rename(columns = {'Accreted Unit Size': 'Unit Size'})
stock_in_agg_margin = stock_in_agg.copy()

Getting Unit Cost based on Unit Size Dict

In [ ]:
# continuation of above REGARDLESS of presence of corrected stock in data
stock_in_agg_margin['unit_cost'] = stock_in_agg_margin['Est Total Cost'] / (stock_in_agg_margin['Unit Size'] * stock_in_agg_margin['Qty'])

stock_in_agg_margin['productname_cleaned'] = stock_in_agg_margin['Product Name'].apply(lambda x: names_cleaning(str(x))) 

# creating a dictionary of product names
product_name_dictionary = stock_in_agg_margin[['productname_cleaned', 'Product Name', 'Unit Size']].copy()
product_name_dictionary = product_name_dictionary.drop_duplicates()
unique_product_names = product_name_dictionary['productname_cleaned'].drop_duplicates().tolist()

# product name and unit size dictionary to be exported
product_name_dictionary2 = product_name_dictionary[['Product Name', 'Unit Size']]

# total cost
total_stock_in_cost = stock_in_agg_margin['Est Total Cost'].sum()

**Invoice Details to Recipe Ingredients Stock-In**

In [ ]:
# init. embeddings for invoice details
stock_in_embeddings = stock_in_embeddings_fn(recipe_ingredients_list)

# get a list of most similar item stocked from recipe and stock-in sheets
most_similar = []
for item in unique_product_names:
    query_embedding = sbert_model.encode(item, convert_to_tensor = True)
    cos_score = util.pytorch_cos_sim(query_embedding, stock_in_embeddings)[0]
    best_match = torch.topk(cos_score, k = 1)
    for idx in best_match[1]:
        most_similar.append(recipe_ingredients_list[idx])

del query_embedding

# stacking into a df
matched_ingredients_stock_in_df = pd.DataFrame({
    'productname_cleaned': unique_product_names,
    'Ingredient': most_similar
    })

matched_ingredients_stock_in_df = matched_ingredients_stock_in_df.merge(product_name_dictionary)
matched_ingredients_stock_in_df.drop('productname_cleaned', axis = 1, inplace = True)
matched_ingredients_stock_in_df = matched_ingredients_stock_in_df[['Product Name', 'Ingredient']]

# bringing in amended crosswalks
matched_ingredients_stock_in_amended_df = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\nov21_reporting\Adrift\stock_in_ingredients_xwalk_amended.csv', encoding = 'utf-8')
matched_recipe_pos_amended_df = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\nov21_reporting\Adrift\recipe_pos_amended.csv', encoding = 'utf-8')

# appending old crosswalk to new
matched_ingredients_list = matched_ingredients_stock_in_amended_df['Product Name'].drop_duplicates().tolist()
new_items = matched_ingredients_stock_in_df.loc[~matched_ingredients_stock_in_df['Product Name'].isin(matched_ingredients_list),]
matched_ingredients_stock_in_amended_df = pd.concat([matched_ingredients_stock_in_amended_df, new_items])
matched_ingredients_stock_in_amended_df.reset_index(inplace = True, drop = True)

matched_pos_list = matched_recipe_pos_amended_df['POS Items'].drop_duplicates().tolist()
new_pos_items = matched_recipe_pos_df.loc[~matched_recipe_pos_df['POS Items'].isin(matched_pos_list),]
matched_recipe_pos_amended_df = pd.concat([matched_recipe_pos_amended_df, new_pos_items])
matched_recipe_pos_amended_df.reset_index(inplace = True, drop = True)
matched_recipe_pos_amended_df.drop_duplicates(inplace = True)

**POS to Recipe Stock-In to Invoice Details**

In [ ]:
# find out items ordered during the period 
pos_sheet_cleaned_ordered = all_pos_cleaned[['Article', 'Number of articles']].copy()
pos_sheet_cleaned_ordered['Article'] = pos_sheet_cleaned_ordered['Article'].str.upper()
pos_sheet_cleaned_ordered = pos_sheet_cleaned_ordered.merge(matched_recipe_pos_amended_df, left_on = 'Article', right_on = 'POS Items')
pos_sheet_cleaned_ordered['Recipe Items'] = pos_sheet_cleaned_ordered['Recipe Items'].str.upper()
pos_sheet_cleaned_ordered = pos_sheet_cleaned_ordered[['Article', 'Number of articles', 'Recipe Items']]

# ingredient stock in crosswalk
ingredient_stockin_recipe_qc = matched_ingredients_stock_in_amended_df.merge(recipe_sheet_df[['Ingredient', 'Ingredient Ordered (if known)']].drop_duplicates(),
                                                                             on = 'Ingredient')

# merge with recipes df to get the recipes for which we have a crosswalk match
recipe_ordered = recipe_sheet_df.merge(pos_sheet_cleaned_ordered,
                                       left_on = 'Food Item (As per POS system)',
                                       right_on = 'Recipe Items')

In [ ]:
recipe_ordered[['Food Item (As per POS system)']].nunique()

In [ ]:
recipe_sheet_df[['Food Item (As per POS system)']].nunique()

In [ ]:
pos_sheet_cleaned_ordered[['Recipe Items']].nunique()

In [ ]:
print(pos_sheet_cleaned_ordered.loc[~pos_sheet_cleaned_ordered['Recipe Items'].isin(recipe_ordered['Food Item (As per POS system)'].drop_duplicates().tolist()), 'Recipe Items'].drop_duplicates().tolist())

**Unmatched Items**

In [ ]:
# calculate quantity consumed by min serving size
## ceiling
recipe_ordered['Quantity Consumed'] = recipe_ordered['Number of articles']*recipe_ordered['Quantity']


# get items that cannot be matched to recipes
unmatched = matched_ingredients_stock_in_amended_df.loc[matched_ingredients_stock_in_amended_df.Ingredient.isna(),]
unmatched = unmatched.merge(stock_in_agg_margin)
unmatched = unmatched[['Product Name', 'Unit', 'Qty', 'Est Total Cost']]
unmatched = unmatched.drop_duplicates()


# get pos items that cannot be matched properly
unmatched_pos = matched_recipe_pos_amended_df.loc[matched_recipe_pos_amended_df['Recipe Items'].isna(), 'POS Items']
unmatched_pos = pd.DataFrame(unmatched_pos)
unmatched_pos = unmatched_pos.rename(columns = {'POS Items': 'Article'})
unmatched_pos = unmatched_pos.merge(all_pos_cleaned[['Article', 'Number of articles', 'Revenue']])

**Margin Calculation**

In [ ]:
# calculate margins
cost_calculation = recipe_ordered[['Food Item (As per POS system)', 'Ingredient', 'Quantity']].copy()
cost_calculation = cost_calculation.merge(matched_ingredients_stock_in_amended_df, on = 'Ingredient')
cost_calculation = cost_calculation.merge(stock_in_agg_margin[['Product Name', 'unit_cost']], on = 'Product Name')
cost_calculation.replace(np.inf, 0, inplace = True)

In [ ]:
## cost 
cost_calculation = cost_calculation.assign(
    constituent_cost = lambda x: x['Quantity'] * x['unit_cost']
    )

# averaging out the cost when there is more than one possible identical ingredient
summarized_cost_calculation = cost_calculation.groupby(['Food Item (As per POS system)', 'Ingredient', 'Quantity']).agg(
    mean_constituent_cost=('constituent_cost', 'mean'))

summarized_cost_calculation = summarized_cost_calculation.reset_index()
summarized_cost_calculation = summarized_cost_calculation.drop_duplicates()

# obtaining COGS
cost_of_goods_sold = summarized_cost_calculation.groupby('Food Item (As per POS system)').sum()

## crosswalk to connect to the POS system
cost_of_goods_sold = cost_of_goods_sold.merge(matched_recipe_pos_amended_df, 
                                              left_on = 'Food Item (As per POS system)', 
                                              right_on = 'Recipe Items')

cost_of_goods_sold = cost_of_goods_sold.merge(all_pos_cleaned[['Article', 'Revenue']],
                                              left_on = 'POS Items',
                                              right_on = 'Article')

summarized_cost_calculation = summarized_cost_calculation.rename(columns = {
    'mean_constituent_cost': 'Cost of Ingredient'
    })

### Unmatched POS

In [63]:
unmatched_pos = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\Output\Round 2\unmatched_pos_articles.csv', encoding = 'utf-8')
unmatched_inventory = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\Output\Round 2\estimated_unused_orders.csv', encoding = 'utf-8')
stock_in = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\InvoiceDetails_Amalgamated.csv', encoding = 'utf-8')

# get drinks categories
drinks_category = ['Groceries', 'Bakery & Pastry Products', 'Alcoholic Beverage', 'Beverage - Alcohol', 'Alcohol Beverage', 'Alc Beverage', 'Non Alcoholic Beverage', 'Beverage - Non Alcohol', 'Beverage - Soft', 'Beverage  - Soft', 'N-Alc Beverage', 'Food']
category_exclusion_split = ['Printing & Stationary', 'Printing and Stationery Supplies', 'Tax Adjustment', 'CAPEX', 'Other', 'Cleaning', 'Cleaning Supplies', 'Kitchen Supplies', 'Discount', 'Guest Supplies', 'Rounding', 'General Supplies', 
                            'Packaging',  'Bar Expenses','Operating Supplies General', 'HR', 'Payroll', 'Cleaning & Chemical', 'Utilities', 'Music & entertainment', 'Payroll & Related Expenses', 'PR & Marketing', 'Payroll Provision (Guest Chefs)',
                            'Transport', 'Accommodation & Air Tickets', 'OS&E - Kitchen', 'OS&E - FOH', 'Supplies Kitchen', 'Supplies Others', 'Supplies Cleaning', 'Provision', 'Accommodation', 'Crockery', 'Small Equipment', 
                            'Departmental Supplies', 'Cleaning', 'Disposables and Chemicals', 'Payroll and HR related',  'Staff Training', 'Paper Supplies', 'Uniforms', 'Passage', 'Travel Other', 'Miscellaneous Expenses', 
                            'VisaVisa MedicalsMedical Lev', 'Linen', 'Equipment Hire', 'Fuel',  'Meals', 'Marketing Expense', 'Managment Fee', 'Legal / Licenses', 'Pre-Opening Printing and Stationery Supplies', 'Pre-Opening - Kitchenware', 
                            'Pre-Opening - Payroll / HR Related', 'Pre-Opening - Accommodation & Air Tickets', 'Pre-Opening - Linen & Uniform', 'Pre-Opening - Legal / Licenses', 'Pre-Opening-PR & Marketing', 'Pre-Opening - IT & Technology', 
                            'Pre-Opening - China/Glass/Silver', 'Pre-Opening - Staff Meal', 'Pre-Opening Expenses', 'Pre-Opening Operating Supplies', 'Pre-Opening Training', 'Pre-Opening-PR & Marketing', 'Pre-Opening Music and Entertain. Expenses',
                            'OE-China', 'OE-Uniform', 'OE-Others', 'OE-Security/ Cleaning', 'OE-Kitchen supplies',  'OE-Music & entertainment', 'OE-Provision', 'FC-Bank Charges', 'OE - Admin - Supplies', 'OE-Glasswares', 'OE-Provision', 
                            'OE-Laundry', 'OE-Supply cleaning', 'OE-Packaging', 'OE-Guest supplies', 'OE-Printing & stationary', 'OE - ADMIN - Printing', 'OE-Others', 'FC-PR & Marketing', 'OE - Admin - Transport', 'FC-IT & Technology',
                            'OE-Bar Expenses', 'OE-Admin - Meal Allocation', 'Task force']
# filter for drinks ordered
stock_in_drinks = stock_in.loc[~stock_in['Category'].isin(category_exclusion_split), 'Product Name'].drop_duplicates().tolist()

# filter for drinks that are in unmatched inventory
unmatched_inventory_drinks = unmatched_inventory.loc[unmatched_inventory['Product Name'].isin(stock_in_drinks),]
#unmatched_inventory_drinks['Cost'] = unmatched_inventory_drinks['Cost'].apply(lambda x: unit_price_adjustment(str(x)))
unmatched_inventory_drinks = unmatched_inventory_drinks.groupby('Product Name').sum()
unmatched_inventory_drinks.reset_index(inplace = True)

# embeddings fn
def stock_in_embeddings_fn(recipe_ingredients_list):
    stock_in_embeddings = sbert_model.encode(recipe_ingredients_list, convert_to_tensor = True)
    return stock_in_embeddings

# get unmatched pos articles
unmatched_pos_articles = unmatched_pos.Article.tolist()
unique_product_names = unmatched_inventory_drinks['Product Name'].drop_duplicates().tolist()

# init embeddings
stock_in_embeddings = stock_in_embeddings_fn(unique_product_names)

most_similar = []
cos = []
for item in unmatched_pos_articles:
    query_embedding = sbert_model.encode(item, convert_to_tensor = True)
    cos_score = util.pytorch_cos_sim(query_embedding, stock_in_embeddings)[0]
    best_match = torch.topk(cos_score, k = 1)
    for idx in best_match[1]:
        most_similar.append(unique_product_names[idx])
    for idx in best_match[0]:
        cos.append(idx)
        
del query_embedding

# stacking into a df
matched_remainder_df = pd.DataFrame({
    'Unmatched Article': unmatched_pos_articles,
    'Most Similar Stock In': most_similar,
    "Score": cos
    })

matched_remainder_df.to_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\Output\Round 2\unmatched_direct_match.csv', index = False)

In [64]:
# import pos data
pos_sheet_df = pd.read_excel(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\POS.xlsx', skiprows = 6, sheet_name = "Revenue per article")
pos_sheet_df = pos_sheet_df.dropna(how = 'all')
all_pos_cleaned = pos_sheet_df.loc[(pos_sheet_df['Article']!= 'Total'),].copy()
all_pos_cleaned['Article'] = all_pos_cleaned.loc[:,'Article'].str.upper()

# total revenue
all_pos_cleaned = all_pos_cleaned.rename(columns = {'Net revenue': 'Revenue'})

In [65]:
matched_remainder_amended_df = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\Output\Round 2\unmatched_direct_match_amended.csv')
matched_remainder_amended_df = matched_remainder_amended_df.drop_duplicates()
# match on amalgamated stock in because it is easier to do the prelim match on unmatched
## and then expand to include the full range because of wine sold by the glass not being properly accounted in recipe

stock_in_drinks_expanded = stock_in.loc[~stock_in['Category'].isin(category_exclusion_split), ['Product Name', 'Qty', 'Subtotal']].copy()
stock_in_drinks_expanded['Subtotal'] = stock_in_drinks_expanded['Subtotal'].apply(lambda x: unit_price_adjustment(str(x)))
stock_in_drinks_expanded = stock_in_drinks_expanded.groupby('Product Name').sum()


matched_remainder_amended_df = matched_remainder_amended_df.merge(stock_in_drinks_expanded, left_on = "Most Similar Stock In", right_on = "Product Name")
matched_remainder_amended_df = matched_remainder_amended_df.merge(all_pos_cleaned[['Article', 'ID', 'Number of articles', 'Revenue']], left_on = 'Unmatched Article', right_on = 'Article')
matched_remainder_amended_df.drop(['Unmatched Article'], axis = 1, inplace = True)

matched_remainder_amended_df = matched_remainder_amended_df.rename(columns = {
    'Most Similar Stock In': 'Product Name',
    'Subtotal': 'Est Total Cost',
    'Article': 'Most Similar Article'
})

# rearrange columns
matched_remainder_amended_df = matched_remainder_amended_df[['Product Name', 'Qty', 'Est Total Cost', 'Most Similar Article', 'ID', 'Number of articles', 'Revenue']].copy()

# export
matched_remainder_amended_df.to_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\CoffeeCulture\Output\Round 2\unmatched_direct_match_amended_final.csv', index = False)


**Farm2Table Workflow**

In [6]:
def stock_in_embeddings_fn(recipe_ingredients_list):
    stock_in_embeddings = sbert_model.encode(recipe_ingredients_list, convert_to_tensor = True)
    return stock_in_embeddings

unmatched_pos_articles = all_pos_cleaned.Article.tolist()

# init. embeddings for invoice details
unique_product_names = stock_in_agg['Product Name'].drop_duplicates().tolist()
stock_in_embeddings = stock_in_embeddings_fn(unmatched_pos_articles)

# get a list of most similar item stocked from recipe and stock-in sheets

most_similar = []
cos = []
for item in unique_product_names:
    query_embedding = sbert_model.encode(item, convert_to_tensor = True)
    cos_score = util.pytorch_cos_sim(query_embedding, stock_in_embeddings)[0]
    best_match = torch.topk(cos_score, k = 1)
    for idx in best_match[1]:
        most_similar.append(unmatched_pos_articles[idx])
    for idx in best_match[0]:
        cos.append(idx)

del query_embedding

# stacking into a df
matched_remainder_df = pd.DataFrame({
    'Unmatched Article': unique_product_names,
    'Most Similar Article': most_similar,
    "Score": cos
    })

matched_remainder_df = matched_remainder_df.loc[matched_remainder_df['Score']>0.5]

ValueError: All arrays must be of the same length

In [11]:
matched_remainder_df.to_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\Output\pos_stock_in_match.csv', index = False)

In [33]:
# QCed Stock In Match
qc_stock_in_match_farm_2_table = pd.read_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\Output\pos_stock_in_match.csv')

qc_stock_in_match_farm_2_table = qc_stock_in_match_farm_2_table.merge(stock_in_agg[['Product Name', 'Qty', 'Est Total Cost']], how = 'left')

# merge with POS
## import pos data
pos_sheet_df = pd.read_excel(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\POS.xlsx', skiprows = 6, sheet_name = "Revenue per article")
pos_sheet_df = pos_sheet_df.dropna(how = 'all')
all_pos_cleaned = pos_sheet_df.loc[(pos_sheet_df['Article']!= 'Total'),].copy()
all_pos_cleaned['Article'] = all_pos_cleaned.loc[:,'Article'].str.upper()

qc_stock_in_match_farm_2_table = qc_stock_in_match_farm_2_table.merge(all_pos_cleaned[['Article', 'ID', 'Number of articles', 'Net revenue']], left_on = 'Most Similar Article', right_on = 'Article')
qc_stock_in_match_farm_2_table.drop(['Article'], inplace = True, axis = 1)
qc_stock_in_match_farm_2_table = qc_stock_in_match_farm_2_table[['Product Name', 'Qty', 'Est Total Cost', 'Most Similar Article', 'ID', 'Number of articles', 'Net revenue']].copy()

qc_stock_in_match_farm_2_table.to_csv(r'C:\Users\wesch\Documents\FoodRazor\expo\output\dec21_reporting\Farm2Table\Output\pos_stock_in_match_w_pos.csv', index = False)